In [2]:
from pathlib import Path
import os
import json
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# 多分もう使わない

In [27]:
from feature import Feature
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [15]:
def json2data(path, max_ = 300):
    cols = ["text", "label", "subLabel"]
    df = pd.DataFrame(index=[], columns=cols)
    files = os.listdir(path)
    for cop in files:
        if "." not in cop:
            continue
        with open(path+cop, "r") as f:
            json_data = json.load(f)
            mode = cop.split(".")[0]
            for i, data in enumerate(json_data[mode]):
                if i >= max_:
                    break
                text = data["data"]
                label = data["label"][0]
                subLabel = ""
                df = df.append(pd.DataFrame([text, label, subLabel], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [5]:
# label_list = "YN WH please plain".split()
label_list = "YN WH NOT".split()
label_dict = dict( zip(label_list, range(len(label_list))) )
print(label_dict)
def extract_X_y(df):
    X = []
    y = []
    for te, la in zip(df.text, df.label):
        X.append(te)
        if la not in label_dict:
            y.append(0)
        else:
            y.append(1)
    return X, y

{'YN': 0, 'WH': 1, 'NOT': 2}


In [6]:
corpus_root = "../../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path

'../../corpus/question/'

In [18]:
df = json2data(data_path)

In [19]:
X, y = extract_X_y(df)
len(X)

1004

In [20]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [21]:
F = Feature()
F.make_features(X_train_str)

X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

300


In [22]:
lr = LogisticRegression(solver='sag', max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='sag')

In [23]:
y_pred = lr.predict(X_test)

In [35]:
y_log = lr.predict_proba(X_test)

In [37]:
print(label_dict)
for y_p, x_s in zip(y_log[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p[1], x_s))

{'YN': 0, 'WH': 1, 'NOT': 2}
0.7851776097250244 : きみはテレビゲームが好きかい？
0.11460969125396304 : 申し訳ないけど、知りませんね。
0.027222053865762916 : 朝食のサンドイッチを作ってあげるよ。
0.021518936000066543 : はい。ペパローニピザに、シーフードパスタ、それにサラダをお願いします。
0.4088095887224571 : いっしょに来ますか？
0.5033779592808952 : ちょっとアドバイスをもらえませんか。
0.053665666840919016 : 「今朝は駅まで乗せてもらえるかな？」とケンは父親に聞きました。「遅れそうなんだ」
0.532502696628346 : これをSNSに載せてもいい？
0.03593148304770703 : あまねは、モニカにはひそかに好きな人がいるのを知らないんだ。
0.0754320261568931 : エマ・スミスさんをお願いできますか？
0.9117604926698546 : 僕が9時ごろ送ったメールは読んだかい？
0.3500339835439612 : もっと強く引っ張らないと。
0.758447412364085 : 誕生日に何が欲しい？
0.1330649963516219 : このロボットを直してくれませんか？
0.09599947869350231 : 雨が降るときには、ぼくは家の中で座って本を読むのが大好きなんだよ。
0.1056432556522079 : あなたのホストファミリーにあげてね。
0.05284078872971196 : すみませんが。銀行への行き方を教えていただけませんか？
0.20237836512678653 : ちょっと聞き取れなかった。
0.9885450244517617 : 君は何か買ったのかい？
0.9156332066280782 : 目標に達したのかい？
0.47550940443763545 : 先に行かせてもらってもよろしいですか？
0.12742294216270608 : 私のコンピューターをモニターにつないでくれませんか？
0.6381296671913905 : いっしょに行ってもいいかな？
0.670388143

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[159  16]
 [ 29  98]]
accuracy =  0.8509933774834437
precision =  0.8596491228070176
recall =  0.7716535433070866
f1 score =  0.8132780082987552


In [28]:
F_path = "../X_y_data/response/"
F_name = "is_question_F.dill"
featureM = DataManager(F_path, format_="dill")
featureM.save_data(F_name, F)
    

success save : ../X_y_data/response/is_question_F.dill


In [29]:
model_path = "../models/response/"
model_name = "is_question_M.pickle"
modelM = DataManager(model_path)
print(model_name)

is_question_M.pickle


In [30]:
modelM.save_data(model_name, lr)

success save : ../models/response/is_question_M.pickle
